In [1]:
import numpy as np 
import pandas as pd
from category_encoders import TargetEncoder 
import json 


In [3]:
def feature_engineering(df):
    df_completo = pd.read_csv('../../data/Limpio/PreProcesado/completo.csv')
    mean_marcas = df_completo.groupby('Marca')['Precio'].mean()
    mean_modelos = df_completo.groupby('Modelo')['Precio'].mean()
    mean_versiones = df_completo.groupby('Versión')['Precio'].mean()

    # saco las columnas que quiero desestimar 
    df.drop(columns = ['Título', 'Tipo de carrocería', 'Puertas', 'Moneda', 'Motor'], inplace = True)

    encoder = TargetEncoder(cols = ['Marca', 'Modelo', 'Versión', 'Color', 'Tracción', 'Transmisión', 'Tipo de vendedor', 'Tipo de combustible', 'Con cámara de retroceso'], smoothing = 10)
    final_df = encoder.fit_transform(df, df['Precio'])

    # vuelvo a binario las columnas que quedan 
    final_df['Tracción'] = final_df['Tracción'].apply(lambda x: 1 if x == '4X4' else 0)
    final_df['Turbo'] = final_df['Turbo'].apply(lambda x: 1 if x == 'SI' else 0)
    final_df['7plazas'] = final_df['7plazas'].apply(lambda x: 1 if x == 'SI' else 0)

    precio = final_df.pop('Precio')
    final_df['Precio'] = precio
    return final_df.values

df = pd.read_csv('../../data/Limpio/PreProcesado/train.csv')
df_encoded = feature_engineering(df)
df_encoded.head()



,Marca,Modelo,Año,Versión,Color,Tipo de combustible,Transmisión,Kilómetros,Precio,Tipo de vendedor,Con cámara de retroceso,Cilindrada,Tracción,Turbo,7plazas
0,21994.112403,20198.785858,2023.0,30978.771349,30615.653691,25490.914073,31986.306034,5000.0,30097.0874,21931.462770,34633.407345,1.2,0,1,0
1,13267.164441,13267.164441,2012.0,14069.507102,22527.662346,25490.914073,31986.306034,106000.0,12000.0000,21931.462770,25599.048801,2.4,0,0,0
2,29625.751163,23016.054624,2020.0,27102.331758,22527.662346,25490.914073,31986.306034,80000.0,21000.0000,21931.462770,25599.048801,1.8,0,0,0
3,29625.751163,23016.054624,2019.0,27621.833433,22527.662346,25490.914073,31986.306034,62000.0,22000.0000,26841.756791,25599.048801,1.8,0,1,0
4,13267.164441,13267.164441,2011.0,14069.507102,28573.656459,25490.914073,31986.306034,138000.0,10990.0000,26841.756791,25599.048801,2.4,0,0,1


In [3]:
def feature_engineering_train(df, dict_name):
    # por cada elemento de cada columna, se le calcula el promedio y se lo guarda en un diccionario 
    mean_encoders = {} 
    mean = df['Precio'].mean()
    for col in ['Marca', 'Modelo', 'Versión', 'Color', 'Transmisión', 'Tipo de vendedor', 'Tipo de combustible', 'Con cámara de retroceso']:
        mean_encoders[col] = mean
        for i in df[col].unique():
            aux_mean = df[df[col] == i]['Precio'].mean()
            mean_encoders[i] = aux_mean

    # guardo el diccionario en un archivo csv donde la primer columna son los keys y la segunda los values 
    with open(dict_name, 'w') as file:
        json.dump(mean_encoders, file) 

    encoder = TargetEncoder(cols = ['Marca', 'Modelo', 'Versión', 'Color', 'Transmisión', 'Tipo de vendedor', 'Tipo de combustible', 'Con cámara de retroceso'], smoothing = 10)
    final_df = encoder.fit_transform(df, df['Precio'])
    return final_df


def feature_engineering_test(df, dict_name):
    # Levantar el diccionario con los promedios desde el archivo JSON
    with open(dict_name, 'r') as file:
        mean_encoders = json.load(file)

    # Aplicar la transformación a cada columna especificada
    for col in ['Marca', 'Modelo', 'Versión', 'Color', 'Transmisión', 'Tipo de vendedor', 'Tipo de combustible', 'Con cámara de retroceso']:
        # Iterar sobre las filas de la columna y aplicar la codificación
        df[col] = df[col].apply(lambda x: mean_encoders[x] if x in mean_encoders else mean_encoders[col])

    return df

def feature_engineering(df, train):
    # saco las columnas que quiero desestimar 
    df.drop(columns = ['Título', 'Tipo de carrocería', 'Puertas', 'Moneda', 'Motor'], inplace = True)

    final_df = df.copy() 

    # vuelvo a binario las columnas que quedan 
    final_df['Tracción'] = final_df['Tracción'].apply(lambda x: 1 if x == '4X4' else 0)
    final_df['Turbo'] = final_df['Turbo'].apply(lambda x: 1 if x == 'SI' else 0)
    final_df['7plazas'] = final_df['7plazas'].apply(lambda x: 1 if x == 'SI' else 0)

    dict_name = "./src/ingeniera_datos/mean_encoders.json" 

    if train:
        final_df = feature_engineering_train(final_df, dict_name)
    else:
        final_df = feature_engineering_test(final_df, dict_name) 

    precio = final_df.pop('Precio')
    final_df['Precio'] = precio

    return final_df

# df = pd.read_csv('../../data/Limpio/PreProcesado/train.csv')
# df_encoded = feature_engineering(df, True)
# df_encoded.head()

df = pd.read_csv('../../data/Limpio/PreProcesado/test.csv')
df_encoded = feature_engineering(df, False)
df_encoded.head()


,Marca,Modelo,Año,Versión,Color,Tipo de combustible,Transmisión,Kilómetros,Tipo de vendedor,Con cámara de retroceso,Cilindrada,Tracción,Turbo,7plazas,Precio
0,23140.533683,15274.418532,2021.0,16452.261522,20390.198235,25490.914073,16183.554592,42000.0,26841.756791,25599.048801,1.6,0,0,0,19417.4757
1,23140.533683,15274.418532,2020.0,16512.961170,30615.653691,25490.914073,31986.306034,31908.0,26841.756791,25599.048801,1.6,0,1,0,20000.0000
2,23140.533683,15274.418532,2019.0,16452.261522,28697.796147,25490.914073,16183.554592,80000.0,26841.756791,25599.048801,1.6,0,0,0,18350.0000
3,23140.533683,15274.418532,2017.0,16452.261522,25993.592240,25490.914073,16183.554592,71600.0,21931.462770,25599.048801,1.6,0,0,0,13000.0000
4,23140.533683,15274.418532,2017.0,15092.701143,25993.592240,25490.914073,16183.554592,115000.0,26841.756791,25599.048801,1.6,0,0,0,10679.6117
